In [ ]:
import boto3

In [ ]:
def get_job_details(job_name):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('jobs')
    job_details = table.get_item(Key={'job_id': job_name})['Item']
    return job_details

In [ ]:
job_details = get_job_details('ghactivity_ingest')
job_details

In [ ]:
baseline_days = job_details['baseline_days']
baseline_days

In [ ]:
from datetime import datetime as dt

In [ ]:
from datetime import timedelta as td

In [ ]:
start_time = dt.now().date() - td(days=int(baseline_days))
start_time

In [ ]:
start_file = f"{dt.strftime(start_time, '%Y-%m-%d')}-0.json.gz"

In [ ]:
start_file

In [ ]:
job_details = {
    'job_id': 'ghactivity_ingest',
    'job_description': 'Ingest ghactivity data to s3',
    'is_active': 'Y',
    'baseline_days': 3,
    'job_run_bookmark_details': {
        'last_run_file_name': start_file,
    }
}

In [ ]:
dynamodb = boto3.resource('dynamodb')

In [ ]:
table = dynamodb.Table('jobs')

In [ ]:
table.put_item(Item=job_details)

In [ ]:
table.get_item(Key={'job_id': 'ghactivity_ingest'})

In [ ]:
job_run_bookmark_details = table.get_item(Key={'job_id': 'ghactivity_ingest'})['Item']['job_run_bookmark_details']

In [ ]:
job_run_bookmark_details

In [ ]:
dt_part = job_run_bookmark_details['last_run_file_name'].split('.')[0]

In [ ]:
next_file = f"{dt.strftime(dt.strptime(dt_part, '%Y-%m-%d-%H') + td(hours=1), '%Y-%m-%d-%-H')}.json.gz"

In [ ]:
next_file

In [ ]:
import requests

In [ ]:
res = requests.get(f'https://data.gharchive.org/{next_file}'). \
    content

In [ ]:
file = open(f'data/{next_file}', 'wb')

In [ ]:
file.write(res)

In [ ]:
file.close()

In [ ]:
!ls -ltr data/

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json('data/2022-05-31-1.json.gz', lines=True, orient='records')

In [ ]:
df